# ECDF Optimization 

In [1]:
# Libraries
import pandas as pd 
import numpy as np
from scipy import stats
from scipy.stats import weibull_min,gamma, gengamma, invgamma,halfnorm,halfgennorm,rayleigh,erlang
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import genextreme
from scipy import integrate
import matplotlib.pyplot as plt
from pynverse import inversefunc
import scipy.stats as stats
from scipy.stats import uniform
from scipy.optimize import root_scalar

/home/tiago/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [27]:
#density function
#y: data set
def dg(y, theta1, theta2, theta3, theta4, theta5, theta6):
    c_theta = NumIntH(theta1, theta2, theta3, theta4, theta5, theta6)
    return (1./c_theta)*(y**theta6)*np.exp(-theta1*y-((theta2*y**theta3)+theta4)**theta5)

#log-likelihood function
#x0: data set
def llf(x, t1, t2, t3, t4, t5, t6):

    x0=x[dg(x, t1, t2, t3, t4, t5, t6)>0]

    return (sum(np.log(dg(x0, t1, t2, t3, t4, t5, t6))))

def eval_fit_extreme(samples, parameters_tuple, no_print=True):
        
    logLik = llf(samples, parameters_tuple[0], parameters_tuple[1], parameters_tuple[2], parameters_tuple[3], parameters_tuple[4], parameters_tuple[5])

    k = len(parameters_tuple)
    aic = 2*k - 2*(logLik)
    n = np.log(len(samples))
    aicc = aic + 2*k*(k+1)/(n-k-1)

    bic = k*n-2*logLik
    edc = -2*logLik-k*np.log(n)

    if(no_print):
        print('AIC: ', aic, 'AICc:', aicc, 'BIC: ', bic, 'EDC:', edc,'Log-Likelihood', logLik)
    
    return aic, aicc, bic, edc, logLik

# Gamma

In [4]:
X = [620,470,260,89,388,242, 103,100,39,460,284,1285, 218,393,106,158,152,477, 403,103,69,158,818,947, 399,1274,32,12,134,660, 548,381,203,871,193,531, 317,85,1410,250,41,1101, 32,421,32,343,376,1512, 1792,47,95,76,515,72, 1585,253,6,860,89,1055, 537,101,385,176,11,565, 164,16,1267,352,160,195, 1279,356,751,500,803,560, 151,24,689,1119,1733,2194, 763,555,14,45,776,1, 1747,945,12,1453,14,150, 20,41,35,69,195,89, 1090,1868,294,96,618,44, 142,892,1307,310,230,30, 403,860,23,406,1054,1935, 561,348,130,13,230,250, 317,304,79,1793,536,12, 9,256,201,733,510,660, 122,27,273,1231,182,289, 667,761,1096,43,44,87, 405,998,1409,61,278,407, 113,25,940,28,848,41, 646,575,219,303,304,38, 195,1061,174,377,388,10, 246,323,198,234,39,308, 55,729,813,1216,1618,539, 6,1566,459,946,764,794, 35,181,147,116,141,19, 380,609,546]

print(len(X))

data2 = X

201


In [5]:
import itertools

def expandgrid(*itrs): # https://stackoverflow.com/a/12131385/1100107
    """
    Cartesian product. Reversion is for compatibility with R.
    
    """
    product = list(itertools.product(*reversed(itrs)))
    return [[x[i] for x in product] for i in range(len(itrs))][::-1]

In [6]:
from hyperopt import fmin, tpe, hp, early_stop 

space = {"t1":hp.uniform("t1", 0, 10),
    "t2":hp.uniform("t2", 0, 10),
    "t3":hp.uniform("t3", -10, 10),
    "t4":hp.uniform("t4", 0, 10),
    "t5":hp.uniform("t5", -10, 10),
    "t6":hp.uniform("t6", 0, 50)
}

from hyperopt.fmin import generate_trials_to_calculate

In [7]:
# a Tuple
a1 = [0.045]
a2 = [1]
a3 = [-1.284]
a4 = [0]
a5 = [5]
a6 = [0]


n = [50]

#matriz de parametros
m_par = expandgrid(a1, a2, a3, a4, a5, a6, n)

print(m_par)

print(m_par[1][0], len(m_par))

lines = len(m_par[0])
columns = len(m_par)

for i in range(lines):
    for j in range(columns):
        print(m_par[j][i])
    print('\n')

[[0.045], [1], [-1.284], [0], [5], [0], [50]]
1 7
0.045
1
-1.284
0
5
0
50




In [28]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.integrate import quad

# Funções fornecidas
def H_teta(a1, a2, a3, a4, a5, a6):
    return quad(lambda x: (x**a6)*np.exp(-a1*x - (a2*x**a3 + a4)**a5), 0, np.inf)[0]

def pdf_HG(y, par):
    t1, t2, t3, t4, t5, t6 = par
    H_val = H_teta(t1, t2, t3, t4, t5, t6)
    y = np.array(y)
    pdfh = 1 / H_val * y**t6 * np.exp(-t1 * y - (t2 * y**t3 + t4)**t5)
    return pdfh

def NumIntH(a1, a2, a3, a4, a5, a6):
    # print('theta6= ', a6)
    return quad(lambda x: (x**a6)*np.exp(-a1*x - (a2*x**a3 + a4)**a5), 0, np.inf)[0]

def NumIntH_distribution(a1, a2, a3, a4, a5, a6, y):
    return quad(lambda x: (x**a6)*np.exp(-a1*x - (a2*x**a3 + a4)**a5), 0, y)[0]

def distribution_extreme_H(x, par):
    t1, t2, t3, t4, t5, t6 = par
    c_theta = NumIntH(t1, t2, t3, t4, t5, t6)
   
    cdf_vec = [0.]*len(x)
    
    for index, val in enumerate(x):
        if val==0:
            cdf_vec[index] = 0
        elif np.isnan(val):
            cdf_vec[index] = 0
        else:
            # print('val =', val)
            cdf_vec[index] = (1./c_theta) * NumIntH_distribution(t1, t2, t3, t4, t5, t6, val)

    return cdf_vec

# Função para calcular a ECDF empírica
def ecdf(data):
    x = np.sort(data)
    y = np.arange(1, len(data) + 1) / len(data)
    return x, y

# Função para calcular o erro entre a ECDF empírica e a CDF teórica
def ecdf_error(params, data):
    x_empirical, y_empirical = ecdf(data)
    y_theoretical = distribution_extreme_H(x_empirical, params)
    error = np.sum((y_empirical - y_theoretical) ** 2)
    return error

In [20]:
def extreme_H_fit_box(sample_data, space):

    global data

    data = sample_data
    
    best = fmin(lambda x: ecdf_error_hyperopt(x),
        space=space,
        algo=tpe.suggest,
        max_evals=10000, early_stop_fn=early_stop.no_progress_loss(1000) )
        # max_evals=10000, early_stop_fn=early_stop.no_progress_loss(1000), trials=trials)

    print(best)
    
    optimal_params = list(best.values())
    return optimal_params

def extreme_H_fit_box_trials(sample_data, space, trials):

    global data

    data = sample_data
    
    best = fmin(lambda x: ecdf_error_hyperopt(x),
        space=space,
        algo=tpe.suggest,
        max_evals=10000, early_stop_fn=early_stop.no_progress_loss(1000), trials=trials)

    print(best)
    
    optimal_params = list(best.values())
    return optimal_params

In [10]:
def distribution_extreme_H_dict(x, par):
    t1, t2, t3, t4, t5, t6 = par
    c_theta = NumIntH(t1, t2, t3, t4, t5, t6)

    # ATENÇÃO
    # o Hyperopt está gerando combinações de parâmetros que levam c_theta==0.
    # resolvi explodir o retorno y_theoretical -> inf) para lidar com a exceção.
    if(c_theta==0):
        # print(par)
        return np.inf
        
    
    cdf_vec = [0.]*len(x)
    
    for index, val in enumerate(x):
        cdf_vec[index] = (1./c_theta) * NumIntH_distribution(t1, t2, t3, t4, t5, t6, val)

    return cdf_vec

# Função para calcular o erro entre a ECDF empírica e a CDF teórica
def ecdf_error_dict(params, data):
    x_empirical, y_empirical = ecdf(data)
    y_theoretical = distribution_extreme_H_dict(x_empirical, params)
    error = np.sum((y_empirical - y_theoretical) ** 2)
    return error

def ecdf_error_hyperopt(dict_ent):
    t1 = dict_ent['t1']
    t2 = dict_ent['t2']
    t3 = dict_ent['t3']
    t4 = dict_ent['t4']
    t5 = dict_ent['t5']
    t6 = dict_ent['t6']
    # print(data)
    return ecdf_error_dict([t1, t2, t3, t4, t5, t6], data)

In [11]:
from hyperopt import fmin, tpe, hp, early_stop 

from hyperopt.fmin import generate_trials_to_calculate

data = []

def ecdf_error_hyperopt(dict_ent):
    t1 = dict_ent['t1']
    t2 = dict_ent['t2']
    t3 = dict_ent['t3']
    t4 = dict_ent['t4']
    t5 = dict_ent['t5']
    t6 = dict_ent['t6']
    # print(data)
    return ecdf_error_dict([t1, t2, t3, t4, t5, t6], data)

def extreme_H_fit(sample_data):

    global data

    data = sample_data

    # theta4 and theta5 box
    space = {"t1":hp.uniform("t1", 0, 10),
        "t2":hp.uniform("t2", 0, 10),
        "t3":hp.uniform("t3", -10, 10),
        "t4":hp.uniform("t4", 0, 0.01),
        "t5":hp.uniform("t5", 0.99, 1.01),
        "t6":hp.uniform("t6", -10, 10)
    }

    best = fmin(lambda x: ecdf_error_hyperopt(x),
        space=space,
        algo=tpe.suggest,
        max_evals=10000, early_stop_fn=early_stop.no_progress_loss(1000) )
    
    print(best)
    
    optimal_params = list(best.values())
    return optimal_params

In [21]:
rounds = 2

lines = 1

optimal_params_vec = []


for i in range(lines):
    a1 = m_par[0][i]
    a2 = m_par[1][i]
    a3 = m_par[2][i]
    a4 = m_par[3][i]
    a5 = m_par[4][i]
    a6 = m_par[5][i]
    n  = m_par[6][i]

    print('i =', i)
    
    sample = data2

    # Frechet: fixando theta1 e theta4 (Frechet)
    space = {"t1":hp.uniform("t1", 0, 0.01),
        "t2":hp.uniform("t2", 0, 10),
        "t3":hp.uniform("t3", -10, 10),
        "t4":hp.uniform("t4", 0, 0.01),
        "t5":hp.uniform("t5", -10, 10),
        "t6":hp.uniform("t6", -10, 10)
    }

    # from Gamma

    shape= 0.8274221404636326
    beta= 0.0017845959485496782
    reference_params = [beta, 0, 1, 0, 1, shape-1]
    trials = generate_trials_to_calculate([{'t1' : reference_params[0], 't2' : reference_params[1], 't3' : reference_params[2], 't4' : reference_params[3], 't5': reference_params[4], 't6' : reference_params[5]}])

    for j in range(rounds):  
        # optimal_params = extreme_H_fit_box(sample, space)
        optimal_params = extreme_H_fit_box_trials(sample, space, trials)
        optimal_params_vec.append(optimal_params)
        print(optimal_params)

i = 0
  0%|                                            | 3/10000 [00:00<07:57, 20.94trial/s, best loss: 0.053736574183174596]

/tmp/ipykernel_75028/656181929.py:19: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  return quad(lambda x: (x**a6)*np.exp(-a1*x - (a2*x**a3 + a4)**a5), 0, np.inf)[0]

/tmp/ipykernel_75028/656181929.py:22: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  return quad(lambda x: (x**a6)*np.exp(-a1*x - (a2*x**a3 + a4)**a5), 0, y)[0]

/tmp/ipykernel_75028/656181929.py:22: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return quad(lambda x: (x**a6)*np.exp(-a1*x - (a2*x**a3 + a4)**a5), 0, y)[0]



  0%|                                           | 12/10000 [00:00<11:09, 14.92trial/s, best loss: 0.053736574183174596]

/tmp/ipykernel_75028/656181929.py:19: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return quad(lambda x: (x**a6)*np.exp(-a1*x - (a2*x**a3 + a4)**a5), 0, np.inf)[0]



  0%|▏                                          | 44/10000 [00:02<09:22, 17.70trial/s, best loss: 0.053736574183174596]

...

  2%|▋                                         | 171/10000 [00:10<13:48, 11.87trial/s, best loss: 0.053736574183174596]

/tmp/ipykernel_75028/656181929.py:22: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return quad(lambda x: (x**a6)*np.exp(-a1*x - (a2*x**a3 + a4)**a5), 0, y)[0]



  6%|██▌                                       | 615/10000 [00:40<11:26, 13.66trial/s, best loss: 0.053736574183174596]

/tmp/ipykernel_75028/656181929.py:19: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return quad(lambda x: (x**a6)*np.exp(-a1*x - (a2*x**a3 + a4)**a5), 0, np.inf)[0]



 19%|████████                                  | 1928/10000 [02:10<09:06, 14.78trial/s, best loss: 0.04858418734799191]
{'t1': np.float64(0.0013959587495859867), 't2': np.float64(7.524374651079849), 't3': np.float64(2.1376126910902986), 't4': np.float64(0.0013014252196971883), 't5': np.float64(-0.9807469108467597), 't6': np.float64(-0.28607919974964496)}
[np.float64(0.0013959587495859867), np.float64(7.524374651079849), np.float64(2.1376126910902986), np.float64(0.0013014252196971883), np.float64(-0.9807469108467597), np.float64(-0.28607919974964496)]
 33%|█████████████▌                           | 3323/10000 [02:02<09:45, 11.41trial/s, best loss: 0.023675049361067756]
{'t1': np.float64(0.0014600180926046908), 't2': np.float64(7.168558250732656), 't3': np.float64(-6.554152619274005), 't4': np.float64(0.0017861406540565223), 't5': np.float64(8.57484235581811), 't6': np.float64(-0.29244892768030395)}
[np.float64(0.0014600180926046908), np.float64(7.168558250732656), np.float64(-6.5541526

In [29]:
# from Gamma

shape= 0.8274221404636326
beta= 0.0017845959485496782
reference_params = [beta, 0, 1, 0, 1, shape-1]

# optimized
# reference_params = [1.38873405e-03, 2.03562041e+00, 4.47499151e-01, 0.00000000e+00, 2.86424926e-01, 0.00000000e+00]

a1 = np.float64(reference_params[0])
a2 = np.float64(reference_params[1])
a3 = np.float64(reference_params[2])
a4 = np.float64(reference_params[3])
a5 = np.float64(reference_params[4])
a6 = np.float64(reference_params[5])


data=np.array(data2)

# print('data =', data)

for tuple in optimal_params_vec:
    
    optimal_params = tuple

    print("Reference parameters:", reference_params)
    
    # Imprimir os parâmetros otimizados
    print("Optimal parameters:", optimal_params)   
    
    # referencia
    print('reference (special case)')
    aic, aicc, bic, edc, logLik = eval_fit_extreme(data, [a1, a2, a3, a4, a5, a6])
    
    print('optimized')
    aic, aicc, bic, edc, logLik = eval_fit_extreme(data, optimal_params)

    print('\n')

Reference parameters: [0.0017845959485496782, 0, 1, 0, 1, -0.1725778595363674]
Optimal parameters: [np.float64(0.0013959587495859867), np.float64(7.524374651079849), np.float64(2.1376126910902986), np.float64(0.0013014252196971883), np.float64(-0.9807469108467597), np.float64(-0.28607919974964496)]
reference (special case)
AIC:  2876.8516235519837 AICc: 2827.343612124952 BIC:  2896.6714530003383 EDC: 2854.84164238931 Log-Likelihood -1432.4258117759919
optimized
AIC:  2878.4856674377747 AICc: 2828.977656010743 BIC:  2898.3054968861293 EDC: 2856.475686275101 Log-Likelihood -1433.2428337188874


Reference parameters: [0.0017845959485496782, 0, 1, 0, 1, -0.1725778595363674]
Optimal parameters: [np.float64(0.0014600180926046908), np.float64(7.168558250732656), np.float64(-6.554152619274005), np.float64(0.0017861406540565223), np.float64(8.57484235581811), np.float64(-0.29244892768030395)]
reference (special case)
AIC:  2876.8516235519837 AICc: 2827.343612124952 BIC:  2896.6714530003383 EDC: